In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import xlnet
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import model_utils

In [4]:
import sentencepiece as spm
from prepro_utils import preprocess_text, encode_ids

sp_model = spm.SentencePieceProcessor()
sp_model.Load('sp10m.cased.v9.model')

True

In [5]:
kwargs = dict(
      is_training=True,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='output-model/config.json')

In [11]:
class Model:
    def __init__(
        self,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.float32, [None, None])
        
        xlnet_model = xlnet.XLNetModel(
            xlnet_config=xlnet_config,
            run_config=xlnet_parameters,
            input_ids=tf.transpose(self.X, [1, 0]),
            seg_ids=tf.transpose(self.segment_ids, [1, 0]),
            input_mask=tf.transpose(self.input_masks, [1, 0]))
        
        output = xlnet_model.get_sequence_output()
        lookup_table = xlnet_model.get_embedding_table()
        lookup_table_2 = xlnet_model.get_embedding_table2()
        
        with tf.variable_scope("model", reuse=tf.AUTO_REUSE):
            with tf.variable_scope('lm_loss'):
                softmax_w = lookup_table
                softmax_w = tf.matmul(softmax_w, lookup_table_2)
                softmax_b = tf.get_variable('bias', [xlnet_config.n_token], dtype=output.dtype,
                                    initializer=tf.zeros_initializer())
                logits = tf.einsum('ibd,nd->ibn', output, softmax_w) + softmax_b

In [12]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())

INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>


In [13]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [14]:
tvars = tf.trainable_variables()

assignment_map = {}
initialized_variable_names = {}

name_to_variable = collections.OrderedDict()
for var in tvars:
    name = var.name
    m = re.match('^(.*):\\d+$', name)
    if m is not None:
        name = m.group(1)
    name_to_variable[name] = var
    
name_to_variable

OrderedDict([('model/transformer/r_w_bias',
              <tf.Variable 'model/transformer/r_w_bias:0' shape=(12, 12, 64) dtype=float32_ref>),
             ('model/transformer/r_r_bias',
              <tf.Variable 'model/transformer/r_r_bias:0' shape=(12, 12, 64) dtype=float32_ref>),
             ('model/transformer/word_embedding/lookup_table',
              <tf.Variable 'model/transformer/word_embedding/lookup_table:0' shape=(32000, 128) dtype=float32_ref>),
             ('model/transformer/word_embedding/lookup_table_2',
              <tf.Variable 'model/transformer/word_embedding/lookup_table_2:0' shape=(128, 768) dtype=float32_ref>),
             ('model/transformer/r_s_bias',
              <tf.Variable 'model/transformer/r_s_bias:0' shape=(12, 12, 64) dtype=float32_ref>),
             ('model/transformer/seg_embed',
              <tf.Variable 'model/transformer/seg_embed:0' shape=(12, 2, 12, 64) dtype=float32_ref>),
             ('model/transformer/layer_shared/rel_attn/q/kernel',

In [15]:
checkpoint = 'output-model/model.ckpt-300000'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [16]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from output-model/model.ckpt-300000


In [17]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'alxlnet-base/model.ckpt')

'alxlnet-base/model.ckpt'

In [1]:
!cp output-model/config.json alxlnet-base/config.json
!cp sp10m.cased.v9* alxlnet-base
!tar cvzf alxlnet-base.tar.gz alxlnet-base

alxlnet-base/
alxlnet-base/sp10m.cased.v9.vocab
alxlnet-base/sp10m.cased.v9.model
alxlnet-base/model.ckpt.index
alxlnet-base/config.json
alxlnet-base/model.ckpt.data-00000-of-00001
alxlnet-base/checkpoint
alxlnet-base/model.ckpt.meta
